# Importando os dados de entrada

In [1]:
import pandas as pd

In [2]:
# dados do equipamento externo até a primeira coleta do sd em 09/11/2023
ex1 = pd.read_csv('datalog_ex.csv')
ex1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116090 entries, 0 to 116089
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   data               116090 non-null  object 
 1   hora               116090 non-null  object 
 2   psi                116090 non-null  float64
 3   htu                116090 non-null  float64
 4   tgn                116090 non-null  float64
 5   tbs                116090 non-null  float64
 6   tbu                116090 non-null  float64
 7   itu1               116090 non-null  float64
 8   itu2               116090 non-null  float64
 9   itgu1              116090 non-null  float64
 10  itgu2              116090 non-null  float64
 11  ibutg1             116090 non-null  float64
 12  ibutg2             116090 non-null  float64
 13  or1                116090 non-null  float64
 14  or2                116090 non-null  float64
 15  hpa                116090 non-null  float64
 16  v_

In [3]:
# dados do equipamento externo após a primeira coleta sd em 09/11/2023
ex2 = pd.read_csv('datalog_ex_2.csv')
ex2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7513 entries, 0 to 7512
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   data               7513 non-null   object 
 1   hora               7513 non-null   object 
 2   psi                7513 non-null   float64
 3   htu                7513 non-null   float64
 4   tgn                7513 non-null   float64
 5   tbs                7513 non-null   float64
 6   tbu                7513 non-null   float64
 7   itu1               7513 non-null   float64
 8   itu2               7513 non-null   float64
 9   itgu1              7513 non-null   float64
 10  itgu2              7513 non-null   float64
 11  ibutg1             7513 non-null   float64
 12  ibutg2             7513 non-null   float64
 13  or1                7513 non-null   float64
 14  or2                7513 non-null   float64
 15  hpa                7513 non-null   float64
 16  v_m/s              7513 

In [4]:
# dados do equipamento interno até a primeira coleta do sdem 09/11/2023
interno = pd.read_excel('datalog_in.xlsx')
interno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162387 entries, 0 to 162386
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   data               162387 non-null  object 
 1   hora               162387 non-null  object 
 2   psi                162387 non-null  float64
 3   htu                162387 non-null  float64
 4   tgn                162387 non-null  float64
 5   tbs                162387 non-null  float64
 6   tbu                162387 non-null  float64
 7   itu1               162387 non-null  float64
 8   itu2               162387 non-null  float64
 9   itgu1              162387 non-null  float64
 10  itgu2              162387 non-null  float64
 11  ibutg1             162387 non-null  float64
 12  ibutg2             162387 non-null  float64
 13  or1                162387 non-null  float64
 14  or2                162387 non-null  float64
 15  hpa                162387 non-null  float64
 16  v_

In [5]:
# dados do online do equipamento interno de todo o tempo até hoje 09/02/2024
inOnline = pd.read_csv('online_in.csv')
inOnline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106938 entries, 0 to 106937
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   created_at  106938 non-null  object 
 1   entry_id    106938 non-null  int64  
 2   field1      106938 non-null  float64
 3   field2      106938 non-null  float64
 4   field3      106938 non-null  float64
 5   field4      106938 non-null  float64
 6   field5      106938 non-null  float64
 7   field6      106938 non-null  float64
 8   field7      106938 non-null  float64
 9   field8      106938 non-null  float64
 10  latitude    0 non-null       float64
 11  longitude   0 non-null       float64
 12  elevation   0 non-null       float64
 13  status      106938 non-null  object 
dtypes: float64(11), int64(1), object(2)
memory usage: 11.4+ MB


### Calculando índices no arquivo online

In [6]:
import math
#funções de correção
def orvalho(umidade, temperatura):
    return (243.04 * (math.log(umidade / 100) + ((17.625 * temperatura) / (243.04 + temperatura)))) /\
           (17.625 - math.log(umidade / 100) - ((17.625 * temperatura) / (243.04 + temperatura)))

def itgu(temperaturaDeGlobo, pontoDeOrvalho):
    return temperaturaDeGlobo + (0.36 * pontoDeOrvalho) + 41.5

def ituBovino(umidadeRelativa, temperaturaDeBulboSeco):
    return ((1.8*temperaturaDeBulboSeco) + 32) - ((0.55 - (0.55*(umidadeRelativa/100)))*(((1.8 * temperaturaDeBulboSeco) + 32) - 58))

def corrige(umidade):
    if umidade > 100:
        return 100
    else:
        return umidade

"""def corrigirTabela(esterno1):
    esterno1['umidade'] = esterno1['umidade'].apply(lambda esterno1: corrige(esterno1))
    esterno1['orvalho'] = esterno1.apply(lambda x: orvalho(x['umidade'], x['tbs']), axis=1)
    esterno1['itu'] = esterno1.apply(lambda x: ituBovino(x['umidade'],x['tbs']), axis=1)
    esterno1['itgu'] = esterno1.apply(lambda x: itgu(x['tg'], x['orvalho']), axis=1)
    esterno1['itgup'] = esterno1.apply(lambda x: itgu(x['tgp'], x['orvalho']), axis=1)
    return esterno1
def corrigirTabela2(compost):
    compost['umidadeCB'] = compost['umidadeCB'].apply(lambda esterno1: corrige(esterno1))
    compost['orvalhoCB'] = compost.apply(lambda x: orvalho(x['umidadeCB'], x['tbsCB']), axis=1)
    compost['ituCB'] = compost.apply(lambda x: ituBovino(x['umidadeCB'],x['tbsCB']), axis=1)
    compost['itguCB'] = compost.apply(lambda x: itgu(x['tgCB'], x['orvalhoCB']), axis=1)
    return compost"""

"def corrigirTabela(esterno1):\n    esterno1['umidade'] = esterno1['umidade'].apply(lambda esterno1: corrige(esterno1))\n    esterno1['orvalho'] = esterno1.apply(lambda x: orvalho(x['umidade'], x['tbs']), axis=1)\n    esterno1['itu'] = esterno1.apply(lambda x: ituBovino(x['umidade'],x['tbs']), axis=1)\n    esterno1['itgu'] = esterno1.apply(lambda x: itgu(x['tg'], x['orvalho']), axis=1)\n    esterno1['itgup'] = esterno1.apply(lambda x: itgu(x['tgp'], x['orvalho']), axis=1)\n    return esterno1\ndef corrigirTabela2(compost):\n    compost['umidadeCB'] = compost['umidadeCB'].apply(lambda esterno1: corrige(esterno1))\n    compost['orvalhoCB'] = compost.apply(lambda x: orvalho(x['umidadeCB'], x['tbsCB']), axis=1)\n    compost['ituCB'] = compost.apply(lambda x: ituBovino(x['umidadeCB'],x['tbsCB']), axis=1)\n    compost['itguCB'] = compost.apply(lambda x: itgu(x['tgCB'], x['orvalhoCB']), axis=1)\n    return compost"

In [7]:
inOnline.drop(columns=['entry_id','latitude', 'longitude', 'elevation', 'status'],inplace=True)

In [8]:
inOnline.columns = ['created_at', 'tbs', 'tbu', 'tgn', 'psi', 'htu', 'hpa',
       'itu1', 'v_m/s']

In [9]:
inOnline['or1'] = inOnline.apply(lambda x: orvalho(x['psi'], x['tbs']), axis=1)
inOnline['or2'] = inOnline.apply(lambda x: orvalho(x['htu'], x['tbs']), axis=1)
inOnline['itu1'] = inOnline.apply(lambda x: ituBovino(x['psi'],x['tbs']), axis=1)
inOnline['itu2'] = inOnline.apply(lambda x: ituBovino(x['htu'],x['tbs']), axis=1)
inOnline['itgu1'] = inOnline.apply(lambda x: itgu(x['tgn'], x['or1']), axis=1)
inOnline['itgu2'] = inOnline.apply(lambda x: itgu(x['tgn'], x['or2']), axis=1)

In [10]:
inOnline

,created_at,tbs,tbu,tgn,psi,htu,hpa,itu1,v_m/s,or1,or2,itu2,itgu1,itgu2
0,2023-11-09T15:18:19-03:00,30.09,21.18,30.88,42.50,83.83,982.72,77.255768,0.0,15.959245,27.052457,83.657413,78.125328,82.118884
1,2023-11-09T15:20:25-03:00,29.69,21.05,30.76,43.57,83.68,982.80,76.924964,0.0,15.988420,26.630890,82.978806,78.015831,81.847120
2,2023-11-09T15:21:31-03:00,29.67,21.09,30.67,43.83,83.79,982.80,76.939327,0.0,16.063592,26.633637,82.962618,77.952893,81.758109
3,2023-11-09T15:22:31-03:00,29.63,21.06,30.60,43.91,83.92,982.79,76.901598,0.0,16.056059,26.620837,82.916581,77.880181,81.683501
4,2023-11-09T15:23:37-03:00,29.64,21.09,30.52,43.99,83.95,982.76,76.926080,0.0,16.093620,26.636686,82.937502,77.813703,81.609207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106933,2024-02-09T09:43:30-03:00,28.87,24.39,29.50,67.89,98.00,985.43,79.380275,0.0,22.345868,28.521746,83.680374,79.044512,81.267828
106934,2024-02-09T09:44:36-03:00,28.87,24.49,29.50,68.56,98.00,985.46,79.475959,0.0,22.507433,28.521746,83.680374,79.102676,81.267828
106935,2024-02-09T09:45:36-03:00,28.93,24.50,29.50,68.33,98.00,985.45,79.532300,0.0,22.509327,28.581592,83.787186,79.103358,81.289373
106936,2024-02-09T09:46:42-03:00,29.04,24.56,29.51,68.09,98.00,985.44,79.661133,0.0,22.556238,28.691310,83.983008,79.130246,81.338872


# convertendo as colunas data e hora para datetime

## Explicando...

Para converter duas colunas em um DataFrame Pandas, uma com data no formato DD/MM/YYYY e outra com hora no formato hh:mm:ss, em uma única coluna no formato datetime, siga estas etapas:

**1. Crie uma nova coluna combinando as duas colunas:**

```python
df['datetime'] = pd.to_datetime(df['data'] + ' ' + df['hora'])
```

**Explicação:**

* `df['datetime'] = pd.to_datetime(df['data'] + ' ' + df['hora'])`: Concatena as colunas `data` e `hora` com um espaço em branco e converte a string resultante para o formato datetime.

**Observações:**

* Este método assume que as colunas `data` e `hora` estão no formato especificado. Se as colunas estiverem em outro formato, você precisa convertê-las antes de usar este método.
* Certifique-se de que a ordem das colunas na concatenação seja `data` e `hora`.

**Exemplo:**

```python
df = pd.DataFrame({
    'data': ['01/01/2023', '02/01/2023', '03/01/2023'],
    'hora': ['10:00:00', '11:00:00', '12:00:00']
})

# Criar nova coluna com datetime
df['datetime'] = pd.to_datetime(df['data'] + ' ' + df['hora'])

print(df)
```

**Saída:**

```
       data  hora             datetime
0  01/01/2023  10:00:00 2023-01-01 10:00:00
1  02/01/2023  11:00:00 2023-01-02 11:00:00
2  03/01/2023  12:00:00 2023-01-03 12:00:00
```

**Extensões:**

* Você pode usar outras funções de conversão de data e hora do Pandas para converter as colunas para o formato desejado.
* Você pode usar a função `df.drop()` para remover as colunas `data` e `hora` após criar a coluna `datetime`.

**Dicas:**

* Utilize o método `df.info()` para verificar o tipo de dados das colunas.
* Se você tiver dúvidas sobre o funcionamento do código, consulte a documentação do Pandas: [https://pandas.pydata.org/docs/](https://pandas.pydata.org/docs/): [https://pandas.pydata.org/docs/](https://pandas.pydata.org/docs/).

Espero que este exemplo seja útil!


## Código

In [11]:
ex1['datetime'] = pd.to_datetime(ex1['data'] + ' ' + ex1['hora'],format='%d/%m/%Y %H:%M:%S',utc=True)

In [12]:
ex2['datetime'] = pd.to_datetime(ex2['data'] + ' ' + ex2['hora'],format='%d/%m/%Y %H:%M:%S',utc=True)
interno['datetime'] = pd.to_datetime(interno['data'] + ' ' + interno['hora'],format='%d/%m/%Y %H:%M:%S',utc=True)

In [13]:
inOnline['datetime'] = pd.to_datetime(inOnline['created_at'],utc=True)

# Concatenando tabelas a fim de se obter uma tabela para cada equipamento

## Explicando...

Para concatenar dois DataFrames com as mesmas colunas em Python usando Pandas, você pode usar a função `concat()`.

**Exemplos:**

**1. Concatenar verticalmente (por linha):**

```python
df_combined = pd.concat([df1, df2], axis=0)
```

**2. Concatenar horizontalmente (por coluna):**

```python
df_combined = pd.concat([df1, df2], axis=1)
```

**Parâmetros adicionais:**

* **`ignore_index`:** Se `True`, os índices dos DataFrames não serão preservados na concatenação. O padrão é `False`.
* **`join`:** Define como lidar com linhas com índices diferentes. As opções são `'outer'`, `'inner'`, `'left'`, e `'right'`. O padrão é `'outer'`.
* **`keys`:** Permite especificar nomes para os DataFrames na concatenação.

**Dicas:**

* Verifique se os DataFrames a serem concatenados possuem as mesmas colunas.
* Utilize o método `df.info()` para verificar o índice e as colunas do DataFrame após a concatenação.

**Exemplo:**

```python
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'C': [7, 8, 9], 'D': [10, 11, 12]})

# Concatenar verticalmente
df_combined_v = pd.concat([df1, df2], axis=0)

# Concatenar horizontalmente
df_combined_h = pd.concat([df1, df2], axis=1)

print(df_combined_v)

# Saída:
#    A  B  C  D
# 0  1  4  7  10
# 1  2  5  8  11
# 2  3  6  9  12

print(df_combined_h)

# Saída:
#    A  B  C  D
# 0  1  4  7
# 1  2  5  8
# 2  3  6  9
```

Espero que esta resposta seja útil!


## Código

In [14]:
# irei concatenar ex1 e ex2 já que ambas não possuem sobreposição de dados
externo = pd.concat([ex1, ex2], axis=0)

In [15]:
# No interno, irei extrair dos dados online entre 09/11/2023 e 09/02/2024
# Depois irei concatenar com os dados obtidos do cartão de memória


In [16]:
inOnline.set_index('created_at',inplace=True)

In [17]:
inOnline = inOnline.loc['2023-11-09':'2024-02-09']

In [18]:
interno.drop(columns=['data', 'hora','CTR_W.m-2', 'TRM_K', 'H_Kcal/Kg ar seco', 'alerta'],inplace=True)

In [19]:
inOnline.reset_index(drop='created_at',inplace=True)

In [20]:
interno_all = pd.concat([interno, inOnline], axis=0)

In [21]:
interno_all.drop(columns=['ibutg1','ibutg2'],inplace=True)

In [22]:
interno_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 268798 entries, 0 to 106410
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   psi       268798 non-null  float64            
 1   htu       268798 non-null  float64            
 2   tgn       268798 non-null  float64            
 3   tbs       268798 non-null  float64            
 4   tbu       268798 non-null  float64            
 5   itu1      268798 non-null  float64            
 6   itu2      268798 non-null  float64            
 7   itgu1     268798 non-null  float64            
 8   itgu2     268798 non-null  float64            
 9   or1       268798 non-null  float64            
 10  or2       268798 non-null  float64            
 11  hpa       268798 non-null  float64            
 12  v_m/s     268798 non-null  float64            
 13  datetime  268798 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(13)
memory usage: 30.8

# Correção da velocidade do vento

**Fatores de correção**
- v <= 1,38 -> 2.5296
- v > 1,38 -> 2.2953

In [23]:
def corrige(velocidade):
    if velocidade > 1.38:
        return velocidade * 2.2953
    else:
        return velocidade * 2.5296

#esterno1['v_m/s'] = esterno1['v_m/s'].apply(lambda esterno1: corrige(esterno1))

In [24]:
interno_all['v_m/s'] = interno_all['v_m/s'].apply(lambda interno_all: corrige(interno_all))

In [25]:
externo['v_m/s'] = externo['v_m/s'].apply(lambda externo: corrige(externo))

# Calculando o TRM E CTR

In [26]:
import sympy as sp

In [58]:
ur = sp.symbols('UR')
t = sp.symbols('tbs')
tg = sp.symbols('tgn')
v = sp.symbols('V')
h_ = 6.7 + 0.243 * t + ((ur/100)*10**(7.5*(t/237.3)+t))
trm_ = 100 * (2.51 * (v**1/2) * ((tg+273.15)-(t+273.15)) + ((tg+273.15)/100)**4)**1/4

In [59]:
trm_

31.375*V*(-tbs + tgn) + 1391.69743907525*(0.00366099212886692*tgn + 1)**4

In [67]:
def trm(m_s:float,tgn:float,tbs:float):
    """
    Faz o cálculo da temperatura radiante média, em k.
    :param m_s: Velocidade do ar em m/s.
    :param tgn: Temperatura de globo negro em ºC.
    :param tbs: Temperatura de bulbo seco em ºC.
    :return: Temperatura radiante média (TRM) em K.
    """
    return 100 * (2.51 * (m_s**1/2) * ((tgn+273.15)-(tbs+273.15)) + ((tgn+273.15)/100)**4)**1/4

def ctr(trm:float):
    """
    Faz o cálculo da entalpia termica radiante.
    :param trm: Temperatura radiante média (TRM) em K.
    :return: Entalpia térmica radiante.
    """
    return (0.0000000567 * ((trm)**4))

def __entalpia__(umidade,tbs):
    return 6.7 + 0.243 * tbs + ((umidade/100)*10**(7.5*(tbs/237.3)+tbs))

In [61]:
externo['TRM_K'] = externo.apply(lambda x: trm(x['v_m/s'], x['tgn'],x['tbs']), axis=1)


In [73]:
externo['CTR_W.m-2'] = externo.apply(lambda x: ctr(x['TRM_K']), axis=1)

In [70]:
externo['H1'] = externo.apply(lambda x: __entalpia__(x['psi'],x['tbs']), axis=1)
externo['H2'] = externo.apply(lambda x: __entalpia__(x['htu'],x['tbs']), axis=1)

In [74]:
externo

,data,hora,psi,htu,tgn,tbs,tbu,itu1,itu2,itgu1,...,or2,hpa,v_m/s,CTR_W.m-2,TRM_K,H_Kcal/Kg ar seco,alerta,datetime,H1,H2
0,01/07/2023,17:22:31,64.00,62.40,29.73,29.39,24.29,79.47,79.23,79.10,...,21.46,1015.16,0.328848,1.118322e+06,2107.394914,NaN,NaN,2023-07-01 17:22:31+00:00,1.333729e+30,1.300385e+30
1,01/07/2023,17:23:31,64.58,62.41,29.61,29.30,24.30,79.43,79.10,79.01,...,21.38,1015.20,0.075888,1.105425e+06,2101.292804,NaN,NaN,2023-07-01 17:23:31+00:00,1.086778e+30,1.050261e+30
2,01/07/2023,17:24:37,65.06,62.57,29.46,29.19,24.27,79.34,78.97,78.86,...,21.31,1015.21,3.060816,1.150349e+06,2122.323917,NaN,NaN,2023-07-01 17:24:37+00:00,8.431026e+29,8.108351e+29
3,01/07/2023,17:25:37,64.90,62.59,29.33,29.01,24.09,79.06,78.72,78.66,...,21.15,1015.22,5.715297,1.211935e+06,2150.176471,NaN,NaN,2023-07-01 17:25:37+00:00,5.484311e+29,5.289107e+29
4,01/07/2023,17:26:43,65.35,62.54,29.20,28.91,24.08,78.98,78.57,78.53,...,21.04,1015.22,0.328848,1.086398e+06,2092.191544,NaN,NaN,2023-07-01 17:26:43+00:00,4.354742e+29,4.167491e+29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7508,06/12/2023,08:31:50,61.61,38.79,35.67,29.65,24.08,79.58,76.15,84.90,...,14.14,983.27,3.060816,3.751129e+06,2851.968600,NaN,NaN,2023-12-06 08:31:50+00:00,2.380982e+30,1.499080e+30
7509,06/12/2023,08:32:56,63.56,38.89,35.44,29.23,24.02,79.27,75.66,84.71,...,13.81,983.29,2.883744,3.631474e+06,2828.947983,NaN,NaN,2023-12-06 08:32:56+00:00,9.057628e+29,5.542026e+29
7510,06/12/2023,08:33:56,63.47,39.07,35.25,29.26,24.04,79.31,75.73,84.53,...,13.91,983.30,2.807856,3.431652e+06,2789.202583,NaN,NaN,2023-12-06 08:33:56+00:00,9.712863e+29,5.978912e+29
7511,06/12/2023,08:35:02,63.18,38.99,35.17,29.45,24.16,79.54,75.95,84.49,...,14.05,983.26,2.984928,3.459525e+06,2794.849105,NaN,NaN,2023-12-06 08:35:02+00:00,1.518320e+30,9.369945e+29


# Reduzindo o intervalo de dados para 5 minutos

## Explicando...

Para extrair um novo DataFrame com a média dos dados de temperatura e umidade a cada 5 minutos, você pode usar o método `resample()` do Pandas. 

**Exemplo:**

```python
# Criar DataFrame de exemplo
df = pd.DataFrame({
    'Data': pd.to_datetime(['2023-03-08 10:00:00', '2023-03-08 10:01:00', '2023-03-08 10:02:00',
                             '2023-03-08 10:03:00', '2023-03-08 10:04:00', '2023-03-08 10:05:00',
                             '2023-03-08 10:06:00', '2023-03-08 10:07:00', '2023-03-08 10:08:00',
                             '2023-03-08 10:09:00']),
    'Temperatura': [20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21.0],
    'Umidade': [50, 51, 52, 53, 54, 55, 56, 57, 58, 59]})

# Calcular a média a cada 5 minutos
df_resampled = df.resample('5T').mean()

# Exibir o novo DataFrame
print(df_resampled)
```

**Saída:**

```
              Data  Temperatura  Umidade
0 2023-03-08 10:00:00  20.250000  52.500000
1 2023-03-08 10:05:00  20.650000  55.500000
```

**Explicação:**

* O método `resample()` recebe como argumento a frequência de resampagem desejada. No caso, `'5T'` indica que a média será calculada a cada 5 minutos.
* A função `mean()` calcula a média dos valores para cada coluna dentro do período de resampagem.
* O novo DataFrame `df_resampled` terá a mesma frequência de resampagem (5T) e conterá a média da temperatura e da umidade para cada período de 5 minutos.

**Observações:**

* Certifique-se de que a coluna `Data` está no formato datetime.
* A frequência de resampagem deve ser compatível com o formato da coluna `Data`.
* Você pode usar outras funções além da `mean` para calcular outras estatísticas, como a soma, o máximo ou o mínimo.

**Dicas:**

* Utilize o método `df.info()` para verificar o tipo de dados da coluna `Data`.
* Utilize o método `df.head()` para visualizar as primeiras linhas do DataFrame original e do DataFrame resam

## Código

In [26]:
externo.drop(columns=['data', 'hora','alerta'],inplace=True)

In [27]:
externo.set_index('datetime',inplace=True,drop=True)

In [28]:
externo_5t = externo.resample('5T').max()

In [29]:
interno_all.set_index('datetime',inplace=True,drop=True)
interno_5t = interno_all.resample('5T').max()

In [30]:
externo_5t.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45400 entries, 2023-07-01 17:20:00+00:00 to 2023-12-06 08:35:00+00:00
Freq: 5T
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   psi                26607 non-null  float64
 1   htu                26607 non-null  float64
 2   tgn                26607 non-null  float64
 3   tbs                26607 non-null  float64
 4   tbu                26607 non-null  float64
 5   itu1               26607 non-null  float64
 6   itu2               26607 non-null  float64
 7   itgu1              26607 non-null  float64
 8   itgu2              26607 non-null  float64
 9   ibutg1             26607 non-null  float64
 10  ibutg2             26607 non-null  float64
 11  or1                26607 non-null  float64
 12  or2                26607 non-null  float64
 13  hpa                26607 non-null  float64
 14  v_m/s              26607 non-null  float64
 15  CTR_W.m-2     

In [31]:
interno_5t.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 64052 entries, 2023-07-01 17:20:00+00:00 to 2024-02-09 02:55:00+00:00
Freq: 5T
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   psi     58605 non-null  float64
 1   htu     58605 non-null  float64
 2   tgn     58605 non-null  float64
 3   tbs     58605 non-null  float64
 4   tbu     58605 non-null  float64
 5   itu1    58605 non-null  float64
 6   itu2    58605 non-null  float64
 7   itgu1   58605 non-null  float64
 8   itgu2   58605 non-null  float64
 9   or1     58605 non-null  float64
 10  or2     58605 non-null  float64
 11  hpa     58605 non-null  float64
 12  v_m/s   58605 non-null  float64
dtypes: float64(13)
memory usage: 6.8 MB


# Apagando colunas e linhas vazias

## Explicando...

Para remover linhas vazias de um DataFrame, você pode usar o método `dropna()`. Este método oferece várias opções para controlar como as linhas vazias são detectadas e excluídas.

**Exemplo:**

```python
# Criar DataFrame de exemplo
df = pd.DataFrame({'coluna_1': [1, 2, np.nan], 'coluna_2': [4, np.nan, 6]})

# Apagar linhas com qualquer valor NaN
df_sem_linhas_vazias = df.dropna()

# Exibir o novo DataFrame
print(df_sem_linhas_vazias)
```

**Saída:**

```
   coluna_1  coluna_2
0        1.0        4.0
```

**Explicação:**

* O método `dropna()` é chamado sem argumentos, o que significa que ele removerá qualquer linha que contenha pelo menos um valor NaN em qualquer coluna.
* O novo DataFrame `df_sem_linhas_vazias` não contém linhas vazias.

**Opções do método `dropna()`:

* `axis=0`: Apagar linhas (eixo 0).
* `axis=1`: Apagar colunas (eixo 1).
* `how='any'`: Apagar linhas com qualquer valor NaN (padrão).
* `how='all'`: Apagar linhas com todos os valores NaN.
* `thresh=n`: Apagar linhas com mais de `n` valores NaN.
* `subset`: Especificar colunas específicas para verificar valores NaN.

**Exemplo com opções:**

```python
# Apagar linhas com todos os valores NaN
df_sem_linhas_vazias = df.dropna(how='all')

# Apagar linhas com mais de 1 valor NaN
df_sem_linhas_vazias = df.dropna(thresh=1)

# Apagar linhas com valores NaN apenas na coluna 1
df_sem_linhas_vazias = df.dropna(subset=['coluna_1'])
```

**Observações:**

* O método `dropna()` não modifica o DataFrame original.
* As linhas excluídas não podem ser recuperadas.
* Certifique-se de que as linhas vazias não contêm dados importantes antes de removê-las.

**Dicas:**

* Utilize o método `df.info()` para verificar a quantidade de linhas e colunas no DataFrame antes e depois de remover as linhas vazias.
* Utilize o método `df.head()` para visualizar as primeiras linhas do DataFrame original e do DataFrame sem linhas vazias.

Espero que este exemplo seja útil!

## Código

In [32]:
externo_5t.drop(columns=['CTR_W.m-2', 'TRM_K','H_Kcal/Kg ar seco'],inplace=True)

In [33]:
interno_5t.dropna(axis=0,inplace=True)

In [34]:
externo_5t.dropna(axis=0,inplace=True)

In [35]:
# Criar coluna de data
interno_5t['Data'] = interno_5t.index.strftime('%d/%m/%Y')

In [36]:
# Criar coluna de horário
interno_5t['Horário'] = interno_5t.index.strftime('%H:%M:%S')

In [37]:
# Criar coluna de data
externo_5t['Data'] = externo_5t.index.strftime('%d/%m/%Y')
# Criar coluna de horário
externo_5t['Horário'] = externo_5t.index.strftime('%H:%M:%S')

In [38]:
externo_5t.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26607 entries, 2023-07-01 17:20:00+00:00 to 2023-12-06 08:35:00+00:00
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   psi      26607 non-null  float64
 1   htu      26607 non-null  float64
 2   tgn      26607 non-null  float64
 3   tbs      26607 non-null  float64
 4   tbu      26607 non-null  float64
 5   itu1     26607 non-null  float64
 6   itu2     26607 non-null  float64
 7   itgu1    26607 non-null  float64
 8   itgu2    26607 non-null  float64
 9   ibutg1   26607 non-null  float64
 10  ibutg2   26607 non-null  float64
 11  or1      26607 non-null  float64
 12  or2      26607 non-null  float64
 13  hpa      26607 non-null  float64
 14  v_m/s    26607 non-null  float64
 15  Data     26607 non-null  object 
 16  Horário  26607 non-null  object 
dtypes: float64(15), object(2)
memory usage: 3.7+ MB


In [39]:
interno_5t.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 58605 entries, 2023-07-01 17:20:00+00:00 to 2024-02-09 02:55:00+00:00
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   psi      58605 non-null  float64
 1   htu      58605 non-null  float64
 2   tgn      58605 non-null  float64
 3   tbs      58605 non-null  float64
 4   tbu      58605 non-null  float64
 5   itu1     58605 non-null  float64
 6   itu2     58605 non-null  float64
 7   itgu1    58605 non-null  float64
 8   itgu2    58605 non-null  float64
 9   or1      58605 non-null  float64
 10  or2      58605 non-null  float64
 11  hpa      58605 non-null  float64
 12  v_m/s    58605 non-null  float64
 13  Data     58605 non-null  object 
 14  Horário  58605 non-null  object 
dtypes: float64(13), object(2)
memory usage: 7.2+ MB


In [40]:
#conda install xlsxwriter

In [41]:
# Create an ExcelWriter object
documento = pd.ExcelWriter('dados.xlsx')

# Save DataFrame to a sheet
interno_5t.to_excel(documento, sheet_name='interno', index=False)
externo_5t.to_excel(documento, sheet_name='externo', index=False)

# Save the Excel file
documento.close()

# Selecionando dias específicos do arquivo

### dados:
**Período chuvoso** <br>
22 de Julho - fisiológico <br>
23 de julho - comportamento <br>
24 de julho- coletas de leite<br>

22 de agosto - coletas de leite<br>
26 de agosto - fisiológico <br>
27 de agosto - comportamento <br>

6 de setembro - coletas de leite<br>
9 de setembro - fisiológico <br>
10 de setembro - comportamento<br>


**Período seco** <br>
4 de dezembro - coletas de leite <br>
9 de dezembro - fisiológico<br>
10 de dezembro comportamento<br>

In [42]:
dados_interno = pd.read_excel('dados.xlsx',sheet_name='interno')
dados_externo = pd.read_excel('dados.xlsx',sheet_name='externo')

In [43]:
lista_de_datas = ['22/07/2023','23/07/2023','24/07/2023','22/08/2023','26/08/2023',
         '27/08/2023','06/09/2023','09/09/2023','10/09/2023','04/12/2023',
         '09/12/2023','10/12/2023']

In [44]:
data = pd.DataFrame({'Data':lista_de_datas})

In [45]:
datafremes_interno = []
datafremes_externo = []

In [46]:
for i in lista_de_datas:
    datafremes_interno.append(dados_interno.loc[dados_interno['Data'] == i])
    datafremes_externo.append(dados_externo.loc[dados_externo['Data'] == i])

In [47]:
nova_tabela_interno = pd.concat(datafremes_interno, axis=0)
nova_tabela_externo = pd.concat(datafremes_externo, axis=0)

In [48]:
nova_tabela_interno

,psi,htu,tgn,tbs,tbu,itu1,itu2,itgu1,itgu2,or1,or2,hpa,v_m/s,Data,Horário
2713,91.18,93.70,20.56,20.06,19.12,67.620000,67.760000,68.750000,86.810000,18.580000,19.010000,991.13,0,22/07/2023,00:00:00
2714,91.18,93.71,20.56,20.06,19.12,67.620000,67.760000,68.750000,86.810000,18.580000,19.020000,991.13,0,22/07/2023,00:05:00
2715,91.58,93.76,20.62,20.19,19.27,67.850000,67.980000,68.870000,86.920000,18.740000,19.150000,991.11,0,22/07/2023,00:10:00
2716,91.21,93.84,20.64,20.19,19.25,67.840000,67.990000,68.870000,86.930000,18.710000,19.160000,991.03,0,22/07/2023,00:15:00
2717,91.30,93.90,20.69,20.19,19.26,67.840000,67.990000,68.930000,87.000000,18.720000,19.170000,991.03,0,22/07/2023,00:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41474,74.84,98.00,24.50,23.87,20.77,72.579777,74.779374,72.853951,74.472394,19.099036,23.534428,983.68,0,10/12/2023,23:35:00
41475,75.54,98.00,24.44,23.81,20.80,72.547443,74.672562,72.837813,74.390848,19.188371,23.474579,983.73,0,10/12/2023,23:40:00
41476,75.94,98.00,24.40,23.75,20.83,72.533473,74.565750,72.801384,74.329303,19.253846,23.414729,983.75,0,10/12/2023,23:45:00
41477,76.31,98.00,24.32,23.69,20.81,72.466308,74.458938,72.752540,74.227757,19.257054,23.354880,983.77,0,10/12/2023,23:50:00


In [49]:
# Create an ExcelWriter object
documento2 = pd.ExcelWriter('datas.xlsx')
# Save DataFrame to a sheet
nova_tabela_interno.to_excel(documento2, sheet_name='in_selecao', index=False)
nova_tabela_externo.to_excel(documento2, sheet_name='ex_selecao', index=False)

# Save the Excel file
documento2.close()

# Classificação dos dados

![itgu_for_cattle.png](itgu_for_cattle.png) 

In [72]:
def __itgu_classificacao__(itgu):
    if itgu < 74:
        return 'conforto'
    elif itgu < 79:
        return 'alerta'
    elif itgu < 84:
        return 'perigo'
    else:
        return 'emergencia'

![itu_for_cattle.png](itu_for_cattle.png)

In [71]:
def __itu_classificacao__(itu):
    if itu <= 70:
        return 'conforto'
    elif itu <= 78:
        return 'alerta'
    elif itu <= 83:
        return 'perigo'
    else:
        return 'emergencia'

**HORÁRIOS**
- Madrugada
> 00:00:00 -> 05:59:59
- Manhã
> 06:00:00 -> 11:59:59
- Tarde
> 12:00:00 -> 17:59:59
- Noite
> 18:00:00 -> 23:59:59

## Código

In [59]:
from datetime import datetime
manha = '05:59:59'
tarde = '11:59:59'
noite = '17:59:59'
manha_t = datetime.strptime(manha, '%H:%M:%S')
tarde_t= datetime.strptime(tarde, '%H:%M:%S')
noite_t = datetime.strptime(noite, '%H:%M:%S')

In [63]:
def __periodo_classificacao__(horario):
    if horario <= manha_t:
        return 'madrugada'
    elif horario > manha_t and horario <= tarde_t:
        return 'manha'
    elif horario > tarde_t and horario <= noite_t:
        return 'tarde'
    else:
        return 'noite'
    

In [84]:
dados_interno

,psi,htu,tgn,tbs,tbu,itu1,itu2,itgu1,itgu2,or1,or2,hpa,v_m/s,Data,Horário,periodo,itu1_nivel,itu2_nivel,itgu1_nivel,itgu2_nivel
0,72.04,68.20,29.15,27.81,23.99,78.180000,77.680000,78.610000,98.950000,22.190000,21.290000,1017.17,0,01/07/2023,17:20:00,tarde,perigo,alerta,alerta,emergencia
1,72.43,69.04,28.86,27.57,23.89,77.950000,77.440000,78.330000,98.560000,22.130000,21.330000,1017.21,0,01/07/2023,17:25:00,tarde,alerta,alerta,alerta,emergencia
2,84.83,83.92,25.25,24.70,22.85,74.920000,74.820000,74.660000,93.630000,21.970000,21.800000,986.25,0,03/07/2023,10:40:00,manha,alerta,alerta,alerta,emergencia
3,102.15,71.35,27.31,26.68,26.89,80.220000,76.540000,78.520000,97.080000,26.990000,21.050000,982.54,0,12/07/2023,14:10:00,tarde,perigo,alerta,alerta,emergencia
4,105.22,71.22,27.39,26.85,27.36,80.750000,76.790000,78.760000,97.180000,27.600000,21.200000,982.48,0,12/07/2023,14:15:00,tarde,perigo,alerta,alerta,emergencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58600,91.74,98.00,25.69,25.19,24.19,76.463293,77.129238,75.739912,76.136397,23.749757,24.851103,984.85,0,09/02/2024,02:35:00,madrugada,alerta,alerta,alerta,alerta
58601,92.00,98.00,25.65,25.19,24.22,76.490952,77.129238,75.716845,76.096397,23.796790,24.851103,984.85,0,09/02/2024,02:40:00,madrugada,alerta,alerta,alerta,alerta
58602,92.24,98.00,25.62,25.19,24.22,76.490952,77.129238,75.686845,76.066397,23.796790,24.851103,984.76,0,09/02/2024,02:45:00,madrugada,alerta,alerta,alerta,alerta
58603,92.41,98.00,25.62,25.12,24.21,76.413828,77.004624,75.688507,76.041261,23.801410,24.781279,984.62,0,09/02/2024,02:50:00,madrugada,alerta,alerta,alerta,alerta


In [78]:
dados_externo

,psi,htu,tgn,tbs,tbu,itu1,itu2,itgu1,itgu2,ibutg1,...,hpa,v_m/s,Data,Horário,hora_t,periodo,itu1_nivel,itu2_nivel,itgu1_nivel,itgu2_nivel
0,65.06,62.57,29.73,29.39,24.30,79.47,79.23,79.10,99.70,25.92,...,1015.21,3.060816,01/07/2023,17:20:00,1900-01-01 17:20:00,tarde,perigo,perigo,perigo,emergencia
1,66.14,62.60,29.33,29.01,24.12,79.06,78.72,78.66,99.15,25.67,...,1015.22,5.715297,01/07/2023,17:25:00,1900-01-01 17:25:00,tarde,perigo,perigo,alerta,emergencia
2,74.99,64.39,24.54,23.40,20.42,71.90,70.96,72.77,92.24,21.66,...,1017.48,4.406976,04/07/2023,15:45:00,1900-01-01 15:45:00,tarde,alerta,alerta,conforto,emergencia
3,69.04,59.89,20.94,20.23,16.81,66.63,66.11,67.61,86.78,18.05,...,1017.74,0.000000,04/07/2023,16:15:00,1900-01-01 16:15:00,tarde,conforto,conforto,conforto,emergencia
4,120.15,57.19,33.04,29.74,32.29,88.58,79.05,86.40,105.64,32.51,...,928.22,0.000000,10/07/2023,11:50:00,1900-01-01 11:50:00,manha,emergencia,perigo,emergencia,emergencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26602,68.41,42.96,32.57,28.17,23.56,78.10,74.91,81.70,103.48,26.23,...,983.30,3.947916,06/12/2023,08:15:00,1900-01-01 08:15:00,manha,perigo,alerta,perigo,emergencia
26603,64.74,42.37,33.58,29.43,23.87,79.26,76.09,82.72,104.85,26.70,...,983.31,3.490848,06/12/2023,08:20:00,1900-01-01 08:20:00,manha,perigo,alerta,perigo,emergencia
26604,61.90,40.61,35.24,30.25,24.09,79.95,76.90,84.37,107.11,27.44,...,983.26,3.580668,06/12/2023,08:25:00,1900-01-01 08:25:00,manha,perigo,alerta,emergencia,emergencia
26605,63.56,39.07,35.67,30.23,24.19,80.02,76.75,84.90,107.73,27.63,...,983.30,3.136704,06/12/2023,08:30:00,1900-01-01 08:30:00,manha,perigo,alerta,emergencia,emergencia


In [87]:
nova_tabela_interno.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3029 entries, 2713 to 41478
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   psi          3029 non-null   float64
 1   htu          3029 non-null   float64
 2   tgn          3029 non-null   float64
 3   tbs          3029 non-null   float64
 4   tbu          3029 non-null   float64
 5   itu1         3029 non-null   float64
 6   itu2         3029 non-null   float64
 7   itgu1        3029 non-null   float64
 8   itgu2        3029 non-null   float64
 9   or1          3029 non-null   float64
 10  or2          3029 non-null   float64
 11  hpa          3029 non-null   float64
 12  v_m/s        3029 non-null   int64  
 13  Data         3029 non-null   object 
 14  Horário      3029 non-null   object 
 15  periodo      3029 non-null   object 
 16  itu1_nivel   3029 non-null   object 
 17  itu2_nivel   3029 non-null   object 
 18  itgu1_nivel  3029 non-null   object 
 19  itgu2_n

In [86]:
nova_tabela_externo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2220 entries, 2716 to 26554
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   psi          2220 non-null   float64
 1   htu          2220 non-null   float64
 2   tgn          2220 non-null   float64
 3   tbs          2220 non-null   float64
 4   tbu          2220 non-null   float64
 5   itu1         2220 non-null   float64
 6   itu2         2220 non-null   float64
 7   itgu1        2220 non-null   float64
 8   itgu2        2220 non-null   float64
 9   ibutg1       2220 non-null   float64
 10  ibutg2       2220 non-null   float64
 11  or1          2220 non-null   float64
 12  or2          2220 non-null   float64
 13  hpa          2220 non-null   float64
 14  v_m/s        2220 non-null   float64
 15  Data         2220 non-null   object 
 16  Horário      2220 non-null   object 
 17  periodo      2220 non-null   object 
 18  itu1_nivel   2220 non-null   object 
 19  itu2_ni

In [55]:
dados_interno['hora_t'] = pd.to_datetime(dados_interno['Horário'],format='%H:%M:%S')

In [64]:
dados_interno['periodo'] = dados_interno['hora_t'].apply(lambda x: __periodo_classificacao__(x))

In [73]:
dados_interno['itu1_nivel'] = dados_interno['itu1'].apply(lambda x: __itu_classificacao__(x))
dados_interno['itu2_nivel'] = dados_interno['itu2'].apply(lambda x: __itu_classificacao__(x))

In [75]:
dados_interno['itgu1_nivel'] = dados_interno['itgu1'].apply(lambda x: __itgu_classificacao__(x))
dados_interno['itgu2_nivel'] = dados_interno['itgu2'].apply(lambda x: __itgu_classificacao__(x))

In [77]:
dados_externo['hora_t'] = pd.to_datetime(dados_externo['Horário'],format='%H:%M:%S')
dados_externo['periodo'] = dados_externo['hora_t'].apply(lambda x: __periodo_classificacao__(x))
dados_externo['itu1_nivel'] = dados_externo['itu1'].apply(lambda x: __itu_classificacao__(x))
dados_externo['itu2_nivel'] = dados_externo['itu2'].apply(lambda x: __itu_classificacao__(x))
dados_externo['itgu1_nivel'] = dados_externo['itgu1'].apply(lambda x: __itgu_classificacao__(x))
dados_externo['itgu2_nivel'] = dados_externo['itgu2'].apply(lambda x: __itgu_classificacao__(x))

In [79]:
nova_tabela_interno['hora_t'] = pd.to_datetime(nova_tabela_interno['Horário'],format='%H:%M:%S')
nova_tabela_interno['periodo'] = nova_tabela_interno['hora_t'].apply(lambda x: __periodo_classificacao__(x))
nova_tabela_interno['itu1_nivel'] = nova_tabela_interno['itu1'].apply(lambda x: __itu_classificacao__(x))
nova_tabela_interno['itu2_nivel'] = nova_tabela_interno['itu2'].apply(lambda x: __itu_classificacao__(x))
nova_tabela_interno['itgu1_nivel'] = nova_tabela_interno['itgu1'].apply(lambda x: __itgu_classificacao__(x))
nova_tabela_interno['itgu2_nivel'] = nova_tabela_interno['itgu2'].apply(lambda x: __itgu_classificacao__(x))

In [80]:
nova_tabela_externo['hora_t'] = pd.to_datetime(nova_tabela_externo['Horário'],format='%H:%M:%S')
nova_tabela_externo['periodo'] = nova_tabela_externo['hora_t'].apply(lambda x: __periodo_classificacao__(x))
nova_tabela_externo['itu1_nivel'] = nova_tabela_externo['itu1'].apply(lambda x: __itu_classificacao__(x))
nova_tabela_externo['itu2_nivel'] = nova_tabela_externo['itu2'].apply(lambda x: __itu_classificacao__(x))
nova_tabela_externo['itgu1_nivel'] = nova_tabela_externo['itgu1'].apply(lambda x: __itgu_classificacao__(x))
nova_tabela_externo['itgu2_nivel'] = nova_tabela_externo['itgu2'].apply(lambda x: __itgu_classificacao__(x))

In [83]:
dados_interno.drop('hora_t',axis=1,inplace=True)

In [85]:
dados_externo.drop('hora_t',axis=1,inplace=True)
nova_tabela_interno.drop('hora_t',axis=1,inplace=True)
nova_tabela_externo.drop('hora_t',axis=1,inplace=True)

In [ ]:
# Create an ExcelWriter object
documento2 = pd.ExcelWriter('arquivo.xlsx')
# Save DataFrame to a sheet
nova_tabela_interno.to_excel(documento2, sheet_name='interno_datas', index=False,)
nova_tabela_externo.to_excel(documento2, sheet_name='externo_datas', index=False)
dados_externo.to_excel(documento2, sheet_name='externo', index=False)
dados_interno.to_excel(documento2, sheet_name='interno', index=False)

# Save the Excel file
documento2.close()